In [14]:
import numpy as np
import os.path as osp
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors  # Students: you can use this implementation to find the 
                                                # Nearest-Neigbors

# Students: Default location of saved latent codes per last cell of main.ipynb, change appropriately if
# you saved them in another way.
vanilla_ae_emb_file = 'cs233_gtda_hw4/data/out/pc_ae_latent_codes.npz'
part_ae_emb_file = 'cs233_gtda_hw4/data/out/part_pc_ae_latent_codes.npz'
my_ae_emb_file = 'cs233_gtda_hw4/data/out/part_pc_ae_my_latent_codes_my.npz'

# Load golden distances (pairwise matrix, or corresponding model/part names in golden_names)
golden_part_dist_file = 'cs233_gtda_hw4/data/golden_dists.npz'
golden_data = np.load(golden_part_dist_file, allow_pickle=True)
golden_part_dist = golden_data['golden_part_dist']
golden_names = golden_data['golden_names']
print(len(golden_names))  # models-name/part combinations
print(golden_names[0])
print(golden_data.files)
print(golden_part_dist.shape)
# To load vanilla-AE-embeddings (if False will open those of the 2-branch AE).
mode = "my" # or False

# Load/organize golden part-aware distances.
sn_id_to_parts = defaultdict(list)
id_to_part_loc = dict()

for i, name in enumerate(golden_names):
    # Extract shape-net model ids of golden, map them to their parts.
    sn_id, _, part_id, _, _ = name.split('_')
    sn_id_to_parts[sn_id].append(part_id)
    
    # Map shape-net model id and part_id to location in distance matrix, (the order is the same).
    id_to_part_loc[(sn_id, part_id)] = i

if mode == "vanilla":
    in_d = np.load(vanilla_ae_emb_file)    # Students: assuming you used the numpy.savez
elif mode == "part":
    in_d = np.load(part_ae_emb_file)
else:
    print("load my data")
    in_d = np.load(my_ae_emb_file)
        
latent_codes = in_d['latent_codes']
test_names = in_d['test_names']

# TODO: Use golden distances and matchings to solve question (g)
cum_part_dists = 0.0
common_parts_num = []
cum_euclidean_dists = 0.0
for i, sn_name in enumerate(test_names):
    parts_of_model = set(sn_id_to_parts[sn_name])
    # minimum euclidean distance between latent_codes[i] and any other latent code
    other_latent_code = latent_codes.copy()
    other_latent_code[i] = np.inf
    dist = np.linalg.norm(latent_codes[i] - other_latent_code, axis=1)
    matched_neighbor = test_names[np.argmin(dist)]
    parts_of_neighbor = set(sn_id_to_parts[matched_neighbor])
    # compute the requested distances.
    # Use id_to_part_loc for each model/part combination
    part_common = list(parts_of_model.intersection(parts_of_neighbor))
    part_dist = 0
    for part in part_common:
        loc1 = id_to_part_loc[(sn_name, part)]
        loc2 = id_to_part_loc[(matched_neighbor, part)]
        part_dist += golden_part_dist[loc1, loc2]
    unique_parts_A = list(parts_of_model - parts_of_model.intersection(parts_of_neighbor))
    for part in unique_parts_A:
        loc1 = id_to_part_loc[(sn_name, part)]
        neighbor_part_dists = []
        for part_u in list(parts_of_neighbor):
            loc2 = id_to_part_loc[(matched_neighbor, part_u)]
            neighbor_part_dists.append(golden_part_dist[loc1, loc2])
        part_dist += max(neighbor_part_dists)
    cum_part_dists += part_dist
    common_parts_num.append(len(part_common))
    cum_euclidean_dists += dist[np.argmin(dist)]

print("Cumulative part-aware distance: ", cum_part_dists)
print("Mean euclidean distance: ", cum_euclidean_dists/len(test_names))
print("Mean number of common parts: ", np.mean(common_parts_num))


484
c59cdd1537bd75ddd0818327fc390a5__2__
['golden_part_dist', 'golden_names']
(484, 484)
load my data
Cumulative part-aware distance:  373.9756412655115
Mean euclidean distance:  1.1785803190867106
Mean number of common parts:  3.2066666666666666


## With part-aware embedding
- Cumulative part-aware distance:  450.45022116601467
- Mean euclidean distance:  0.3626336590449015
- Mean number of common parts:  3.1866666666666665

## Without part-aware embedding
- Cumulative part-aware distance:  477.95636761188507
- Mean euclidean distance:  0.19489020749926567
- Mean number of common parts:  3.1333333333333333

## My embedding

- Cumulative part-aware distance:  373.9756412655115
- Mean euclidean distance:  1.1785803190867106
- Mean number of common parts:  3.2066666666666666